In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
with open('/home/jvdzwaan/data/tmp/remove-title-page/aa__001biog01_01-gs.txt') as f:
    gs_lines = f.readlines()

with open('/home/jvdzwaan/data/tmp/remove-title-page/aa__001biog01_01.txt') as f:
    ocr_lines = f.readlines()

In [ ]:
print(len(gs_lines), len(ocr_lines))

In [ ]:
print(''.join(gs_lines[0:5]))

In [ ]:
print(''.join(ocr_lines[0:5]))

In [ ]:
# remove empty lines
gs_lines = [line for line in gs_lines if line.strip() != '']
ocr_lines = [line for line in ocr_lines if line.strip() != '']
print(len(gs_lines), len(ocr_lines))
print(''.join(gs_lines[0:5]))
print('---')
print(''.join(ocr_lines[0:5]))

In [ ]:
gs_txt = ''.join(gs_lines[:100])
ocr_txt = ''.join(ocr_lines[:100])

In [ ]:
%%time
import edlib

res = edlib.align(gs_txt, ocr_txt)

In [ ]:
print(res)

In [ ]:
%%time
import math

num_lines = 100
prev_ld = math.inf

for i in range(num_lines):
    #print(i)
    gs_txt = ''.join(gs_lines[:num_lines]).lower()
    ocr_txt = ''.join(ocr_lines[i:num_lines]).lower()
    res = edlib.align(gs_txt, ocr_txt)
    #print(res)
    ld = res['editDistance']
    if ld > prev_ld:
        print('Stop')
        print('ld:', ld)
        print('i:', i)
        print(''.join(ocr_lines[i-1:i+5]))
        print('---')
        print(''.join(gs_lines[:5]))
        break
    elif ld < prev_ld:
        prev_ld = ld
    #print('---')

In [ ]:
edlib.align('Voorberigt.', 'VOORBERIGT.')

In [ ]:
edlib.align('Voorberigt.'.lower(), 'VOORBERIGT.'.lower())

In [ ]:
# put in function
import codecs
import math
import edlib

def remove_title_page(txt_without_title_page, txt_with_title_page, num_lines=100):
    """Return (ocr) text without title page.
    
    
    """
    with open(txt_without_title_page) as f:
        without_lines = f.readlines()

    with open(txt_with_title_page) as f:
        with_lines = f.readlines()

    # remove empty lines
    #without_lines = [line for line in without_lines if line.strip() != '']
    #with_lines = [line for line in with_lines if line.strip() != '']
    
    prev_ld = math.inf

    for i in range(num_lines):
        without_txt = ''.join(without_lines[:num_lines]).lower()
        with_txt = ''.join(with_lines[i:num_lines]).lower()
        
        res = edlib.align(without_txt, with_txt)
        ld = res['editDistance']
        
        if ld > prev_ld:
            return ''.join(with_lines[i-1:])
        else:
            prev_ld = ld   

In [ ]:
remove_title_page('/home/jvdzwaan/data/tmp/remove-title-page/aa__001biog01_01-gs.txt', 
                  '/home/jvdzwaan/data/tmp/remove-title-page/aa__001biog01_01.txt')

In [ ]:
remove_title_page('/home/jvdzwaan/data/tmp/remove-title-page/aa__001biog01_01.txt', 
                  '/home/jvdzwaan/data/tmp/remove-title-page/aa__001biog01_01-gs.txt')

In [ ]:
import os

from nlppln.utils import get_files, create_dirs, out_file_name
from tqdm import tqdm_notebook as tqdm

failed = []

ocr_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/ocr-with-title-page-filtered/'
gs_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/gs/'
out_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/ocr-without-title-page/'

create_dirs(out_dir)

ocr_files = get_files(ocr_dir)
gs_files = get_files(gs_dir)

print(len(ocr_files), len(gs_files))

try_lines = (100, 500, 1000, 1500, 2000)

for ocr, gs in tqdm(zip(ocr_files, gs_files), total=len(ocr_files)):
    assert(os.path.basename(ocr) == os.path.basename(gs))
    
    for num_lines in try_lines:
        new_ocr_text = remove_title_page(gs, ocr, num_lines=num_lines)
        if new_ocr_text is not None:
            break
    
    out = out_file_name(out_dir, ocr)
    #print(out)
    with open(out, 'w') as f:
        try:
            f.write(new_ocr_text)
        except:
            print(ocr)
            failed.append((ocr, gs))
print(len(failed))